Reading bronze table

In [0]:
df_bronze = spark.table("data_engineering_project.bronze_orders")
df_bronze.display()

Standardizing column names

In [0]:
from pyspark.sql.functions import col
df_silver = df_bronze.select([col(c).alias(c.lower()) for c in df_bronze.columns])
display(df_silver)

In [0]:
from pyspark.sql.functions import regexp_replace, to_date
df_silver = df_silver.withColumn('order_date', to_date(regexp_replace('order_date','/','-'),'yyyy-mm-dd'))
display(df_silver)

In [0]:
df_silver.where(col("id") == '3758').display()

Trimming extra spaces if present

In [0]:
df_silver.printSchema()

In [0]:
from pyspark.sql.functions import to_date,col
df_silver = df_silver.withColumn('order_date',to_date(col('order_date'),'yyyy-MM-dd'))\
                     .withColumn('sales',col('sales').cast('double'))\
                     .withColumn('quantity',col('quantity').cast('double'))\
                     .withColumn('discount',col('discount').cast('double'))

In [0]:
display(df_silver)